# Openlimits Python Example

In [1]:
import openlimits_python
from dotenv import load_dotenv
import os

## Setup

Initialize a new client from API key data. This only works for Nash at the moment.

In [2]:
load_dotenv()
client = openlimits_python.NashClient(os.getenv("NASH_API_SECRET"), os.getenv("NASH_API_KEY"))

## Account and Market queries

You can find the full set of queries supported [here](https://github.com/nash-io/openlimits-python/blob/master/src/lib.rs)

In [3]:
client.get_account_balances(None) # None is for pagination

[{'balance': {'asset': 'qnt',
   'free': '0.0011742091125000000000000',
   'total': '0.0011742091125000000000000'}},
 {'balance': {'asset': 'usdc',
   'free': '7.3432272685285375000000000',
   'total': '7.3432272685285375000000000'}},
 {'balance': {'asset': 'zrx',
   'free': '0.3049423050000000000000000',
   'total': '0.3049423050000000000000000'}},
 {'balance': {'asset': 'trac',
   'free': '7.9597481990625000000000000',
   'total': '7.9597481990625000000000000'}},
 {'balance': {'asset': 'gunthy',
   'free': '5.5814223834375000000000000',
   'total': '5.5814223834375000000000000'}},
 {'balance': {'asset': 'eth',
   'free': '0.0281682628959737500000000',
   'total': '0.1281682628959737500000000'}},
 {'balance': {'asset': 'neo',
   'free': '0.4511866332571250000000000',
   'total': '0.4511866332571250000000000'}},
 {'balance': {'asset': 'gas',
   'free': '0.0129546881250000000000000',
   'total': '0.0129546881250000000000000'}},
 {'balance': {'asset': 'link',
   'free': '2.18302946511059

In [4]:
client.get_order_history("btc_usdc", None)

[{'order': {'id': '1603597218370000000',
   'market_pair': 'btc_usdc',
   'price': '13287.60000000',
   'order_type': {'order_type': 'limit'},
   'client_order_id': None,
   'created_at': 1603597218367,
   'side': {'side': 'sell'},
   'size': '0.000383810',
   'status': {'order_status': 'filled'}}},
 {'order': {'id': '1603557123448000000',
   'market_pair': 'btc_usdc',
   'price': '13074.40000000',
   'order_type': {'order_type': 'limit'},
   'client_order_id': None,
   'created_at': 1603557123446,
   'side': {'side': 'buy'},
   'size': '0.000986660',
   'status': {'order_status': 'filled'}}},
 {'order': {'id': '1603428549898000000',
   'market_pair': 'btc_usdc',
   'price': '12950.10000000',
   'order_type': {'order_type': 'limit'},
   'client_order_id': None,
   'created_at': 1603428549895,
   'side': {'side': 'sell'},
   'size': '0.000393810',
   'status': {'order_status': 'filled'}}},
 {'order': {'id': '1603428353684000001',
   'market_pair': 'btc_usdc',
   'price': '12950.10000000

In [5]:
client.limit_buy("eth_usdc", "0.0171", "350.46")

{'order': {'id': '1603825281736000000',
  'market_pair': 'eth_usdc',
  'price': None,
  'order_type': {'order_type': 'limit'},
  'client_order_id': None,
  'created_at': 1603825281733,
  'side': {'side': 'buy'},
  'size': '0',
  'status': {'order_status': 'pending'}}}

In [6]:
client.get_order("1603825281736000000")

{'order': {'id': '1603825281736000000',
  'market_pair': 'eth_usdc',
  'price': '350.46000000',
  'order_type': {'order_type': 'limit'},
  'client_order_id': None,
  'created_at': 1603825281733,
  'side': {'side': 'buy'},
  'size': '0.017100',
  'status': {'order_status': 'open'}}}

In [7]:
client.cancel_order("eth_usdc", "1603825281736000000")

{'order_canceled': '1603825281736000000'}

In [10]:
client.get_historic_trades("btc_usdc", None) # None is for pagination

[{'trade': {'liquidity': {'liquidity': 'maker'},
   'market_pair': 'btc_usdc',
   'price': '13646.00000000',
   'qty': '0.001827250',
   'order_id': '1603825425306000001',
   'side': {'side': 'sell'},
   'created_at': 1603825426656,
   'fees': '0',
   'id': 'VNpm_PH5k7iu5ywL6cUjeoATHEtHUgehNh0-Y5FLvEA'}},
 {'trade': {'liquidity': {'liquidity': 'maker'},
   'market_pair': 'btc_usdc',
   'price': '13646.00000000',
   'qty': '0.004391000',
   'order_id': '1603825425306000001',
   'side': {'side': 'sell'},
   'created_at': 1603825426082,
   'fees': '0',
   'id': '5EeaJ9trPym5_x1XSRSqLZZ1HzMXbWpmsgdFHUB9rpA'}},
 {'trade': {'liquidity': {'liquidity': 'maker'},
   'market_pair': 'btc_usdc',
   'price': '13646.00000000',
   'qty': '0.004987000',
   'order_id': '1603825425306000001',
   'side': {'side': 'sell'},
   'created_at': 1603825425678,
   'fees': '0',
   'id': 'fy_ZJttSBjG9dLHDboarz-75hXJO0wzgqWFb3U2oxeI'}},
 {'trade': {'liquidity': {'liquidity': 'maker'},
   'market_pair': 'btc_usdc',


## Subscriptions

Subscriptions work by registering a callback function with the client. This callback function will be passed every incoming event on the subscription. The callback executes within a seperate thread managed by the openlimits Rust library. The client instance and other state may be accessed within the callback via declared `global` variables.

In [13]:
state = {"num_events": 0, "events":[]} # a simple example of state

# this will be our callback function
def process_event(event):
    global client # we can use this to access the client and react to events
    global state
    
    state["num_events"] += 1
    state["events"].append(event)

In [14]:
# now we register the callback with the client
client.set_subscription_callback(process_event)

In [15]:
# now we subscribe to the orderbook
# the second param limits the number of events we get back. we set to 0 as it is not used at the moment
client.subscribe({"orderbook":("btc_usdc", 0)}) 

In [16]:
# now we wait a few moments and inspect the state
print(state["num_events"])

25


In [18]:
# here is the latest incoming event
print(state["events"][-1])

{'orderbook': {'asks': [{'ask_or_bid': {'price': '13636.50000000', 'qty': '0.27078008'}}, {'ask_or_bid': {'price': '13707.80000000', 'qty': '0.70000000'}}, {'ask_or_bid': {'price': '13709.30000000', 'qty': '0.00000000'}}], 'bids': [{'ask_or_bid': {'price': '13544.50000000', 'qty': '1.00000000'}}, {'ask_or_bid': {'price': '13555.40000000', 'qty': '0.85000000'}}, {'ask_or_bid': {'price': '13614.10000000', 'qty': '0.02277053'}}, {'ask_or_bid': {'price': '13614.60000000', 'qty': '0.10073450'}}, {'ask_or_bid': {'price': '13622.70000000', 'qty': '0.00367034'}}]}}
